# Pricing Data Analyst Intern Task: Solutions Notebook

In [2]:
#importing dependencies 
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
import numpy as np

## First of all we need to extract our data sets into separate csv files ↓↓↓ :

In [3]:

# Creating a func that extracts the last 3 sheets from an Excel file and saves each sheet as a separate CSV file.
#data excel fiel should be existing in the current directory

def extract_sheets_to_csv(input_file):
    # Read the Excel file
    xls = pd.ExcelFile(input_file)

    # Get the sheet names (last 3 sheets in our case contains the data we need)
    sheet_names = xls.sheet_names[-3:]

    # Extract each sheet to a separate CSV file, and print it's name to make sure it is extracted
    for sheet_name in sheet_names:
        df = pd.read_excel(xls, sheet_name)
        output_file = f"{sheet_name}.csv"
        df.to_csv(output_file, index=False, encoding="utf-8")
        print(f"Saved {output_file}")

if __name__ == "__main__":
    input_excel_file = "Data.xlsx"  
    extract_sheets_to_csv(input_excel_file)


PermissionError: [Errno 13] Permission denied: 'Sales_orders.csv'

## Now we'll Load our datasets into Pandas Dataframes ↓↓↓ :

In [40]:
# Paths to the CSV files (Make sure they are in the same directory as the notebook)
sales_orders_path = 'Sales_orders.csv'
cost_of_goods_path = 'Cost_of_Goods.csv'
product_sales_orders_path = 'Product_sales_order.csv'


# Load the CSV files with UTF-8 encoding and handle thousand separators
sales_orders = pd.read_csv(sales_orders_path, encoding='utf-8', thousands=',')
cost_of_goods = pd.read_csv(cost_of_goods_path, encoding='utf-8')
product_sales_orders = pd.read_csv(product_sales_orders_path, encoding='utf-8', thousands=',')


# Set data types for sales_orders
sales_orders['ORDER_ID'] = sales_orders['ORDER_ID'].astype(int)
# sales_orders['DATE'] = pd.to_datetime(sales_orders['DATE'], format='%B %d, %Y')
sales_orders['DATE'] = pd.to_datetime(sales_orders['DATE'], format='%Y-%m-%d')
sales_orders['DAY_NAME'] = sales_orders['DAY_NAME'].astype(str)
sales_orders['Order_status'] = sales_orders['Order_status'].astype(str)
sales_orders['SALES'] = sales_orders['SALES'].astype(float)


# Set data types for cost_of_goods
cost_of_goods['PRODUCT_ID'] = cost_of_goods['product_id'].astype(int)
cost_of_goods['purchase_price'] = cost_of_goods['purchase_price'].astype(float)
cost_of_goods['selling_price'] = cost_of_goods['selling_price'].astype(float)


# Set data types for product_sales_orders
product_sales_orders['PRODUCT_SALES_ORDER_ID'] = product_sales_orders['PRODUCT_SALES_ORDER_ID'].astype(int)
product_sales_orders['SALES_ORDER_ID'] = product_sales_orders['SALES_ORDER_ID'].astype(int)
product_sales_orders['PRODUCT_ID'] = product_sales_orders['PRODUCT_ID'].astype(int)
product_sales_orders['SALES'] = product_sales_orders['SALES'].astype(float)


## Data Exploration:

In [41]:
# Verify the data types
print(sales_orders.info())
print(cost_of_goods.info())
print(product_sales_orders.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17163 entries, 0 to 17162
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ORDER_ID      17163 non-null  int32         
 1   DATE          17163 non-null  datetime64[ns]
 2   DAY_NAME      17163 non-null  object        
 3   Order_status  17163 non-null  object        
 4   SALES         17162 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int32(1), object(2)
memory usage: 603.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_id      26 non-null     int64  
 1   purchase_price  26 non-null     float64
 2   selling_price   26 non-null     float64
 3   PRODUCT_ID      26 non-null     int32  
dtypes: float64(2), int32(1), int64(1)
memory usage: 860.0 bytes
None
<class 'pand

In [42]:
# Display the first few rows to verify the content
print(sales_orders.head())
print(cost_of_goods.head())
print(product_sales_orders.head())

   ORDER_ID       DATE DAY_NAME Order_status    SALES
0   6387833 2023-10-01      Sun    Delivered   285.50
1   6385549 2023-10-01      Sun    Delivered  1512.25
2   6387475 2023-10-01      Sun    Delivered   197.50
3   6389331 2023-10-01      Sun    Delivered    67.50
4   6390122 2023-10-01      Sun    Delivered   118.00
   product_id  purchase_price  selling_price  PRODUCT_ID
0         415          270.56         281.75         415
1        3495           20.01          21.75        3495
2         152          205.08         212.25         152
3          72          508.79         540.75          72
4         974          134.01         144.25         974
   PRODUCT_SALES_ORDER_ID  SALES_ORDER_ID  PRODUCT_ID   SALES
0                68097611         6460123        2438  390.00
1                68009163         6448262        1087  122.25
2                68097089         6460054         142  166.32
3                68060918         6455556         415  226.50
4                6814492

In [43]:
print(sales_orders.describe())
print(cost_of_goods.describe())
print(product_sales_orders.describe())

           ORDER_ID                           DATE         SALES
count  1.716300e+04                          17163  1.716200e+04
mean   6.444773e+06  2023-10-10 09:19:32.312532736 -1.858798e+02
min    6.383207e+06            2023-10-01 00:00:00 -1.000000e+07
25%    6.414099e+06            2023-10-05 00:00:00  1.590000e+02
50%    6.442612e+06            2023-10-10 00:00:00  2.355000e+02
75%    6.475502e+06            2023-10-16 00:00:00  4.260000e+02
max    6.524009e+06            2023-10-21 00:00:00  9.568750e+03
std    3.585509e+04                            NaN  7.633851e+04
        product_id  purchase_price  selling_price   PRODUCT_ID
count    26.000000    2.600000e+01      26.000000    26.000000
mean   3277.230769    3.846339e+06     213.980769  3277.230769
std    3441.946209    1.961158e+07     142.814161  3441.946209
min      72.000000   -2.000000e+02      21.750000    72.000000
25%     458.500000    1.206600e+02     130.375000   458.500000
50%    1493.500000    1.823300e+02   

## Data Cleaning Process ↓↓↓:

### 1. Cleaning Dataset1 (Sales_orders)


In [44]:
#Identify columns with missing values (NaN or null).
missing_values_columns = sales_orders.isna().any()
sum_missing_values_columns= sales_orders.isna().sum()
columns_with_missing_values = missing_values_columns[missing_values_columns].index.tolist()
print("Columns with missing values:", columns_with_missing_values, " \n ,Sum of missing values: \n",sum_missing_values_columns)

Columns with missing values: ['SALES']  
 ,Sum of missing values: 
 ORDER_ID        0
DATE            0
DAY_NAME        0
Order_status    0
SALES           1
dtype: int64


In [45]:
# Check the ORDER_ID of the Missing SALES value
null_sales_order_ids = sales_orders[sales_orders['SALES'].isna()]['ORDER_ID']

print("ORDER_IDs of records with null value in SALES column:", null_sales_order_ids.tolist())

ORDER_IDs of records with null value in SALES column: [6383668]


In [46]:
# Checking if this point of sale is existing in the product_sales_orders df, using the ORDER_ID [6383668]
#looking for is 6383668 in product_sales_orders
order_id_to_find = 6383668
# Use 'SALES_ORDER_ID' instead of 'ORDER_ID' for filtering
filtered_df = product_sales_orders[product_sales_orders['SALES_ORDER_ID'] == order_id_to_find]

# Check if the resulting DataFrame is empty
if filtered_df.empty:
    print(f"No records found for SALES_ORDER_ID {order_id_to_find}.")
else:
    print("Row for SALES_ORDER_ID 6383668:\n", filtered_df)

Row for SALES_ORDER_ID 6383668:
        PRODUCT_SALES_ORDER_ID  SALES_ORDER_ID  PRODUCT_ID  SALES
10282                67561235         6383668        1070  392.5
15218                67561236         6383668         152  334.0


In [47]:
# it turns out that the order 6383668 involves the sale of more than one product,
#so the Total revenue per this order is the sum of the Total revenue per order per these two products
# this means that we're going to fill the missing value of SALES in Sales_orders with 392.5+334.0

# Step 1: Calculate the sum of SALES for the specific SALES_ORDER_ID
sales_sum = filtered_df['SALES'].sum()

# Step 2: Locate the row in sales_orders with the matching SALES_ORDER_ID
sales_orders.loc[sales_orders['ORDER_ID'] == order_id_to_find, 'SALES'] = sales_sum

# Display the updated sales_orders DataFrame to verify the change
print(sales_orders[sales_orders['ORDER_ID'] == order_id_to_find])

    ORDER_ID       DATE DAY_NAME Order_status  SALES
29   6383668 2023-10-01      Sun    Delivered  726.5


In [48]:
#Double Check the ORDER_ID of the Missing SALES value
null_sales_order_ids = sales_orders[sales_orders['SALES'].isna()]['ORDER_ID']

print("ORDER_IDs of records with null value in SALES column:", null_sales_order_ids.tolist())

ORDER_IDs of records with null value in SALES column: []


In [49]:
# CHecking for outliers in SALES col of sales_orders
column_name = 'SALES'

min_val = sales_orders[column_name].min()
max_val = sales_orders[column_name].max()

print(f"Minimum value in {column_name}: {min_val}")
print(f"Maximum value in {column_name}: {max_val}")

Minimum value in SALES: -10000000.0
Maximum value in SALES: 9568.75


In [50]:
# there is a record in SALES that indicate a minus 10 million of revenue per this order
#let's check this order id
# Locate the order with the minimum sales value and extract the order ID(s) as a list
tenmillion_loss_order_ids = sales_orders.loc[sales_orders['SALES'] == min_val, 'ORDER_ID'].tolist()

print("The order(s) that lost the company 10 million is/are:", tenmillion_loss_order_ids)

The order(s) that lost the company 10 million is/are: [6418568]


In [51]:
#looking for is 6418568 in product_sales_orders
order_id_to_find = 6418568
# Use 'SALES_ORDER_ID' instead of 'ORDER_ID' for filtering
filtered_df = product_sales_orders[product_sales_orders['SALES_ORDER_ID'] == order_id_to_find]

# Check if the resulting DataFrame is empty
if filtered_df.empty:
    print(f"No records found for SALES_ORDER_ID {order_id_to_find}.")
else:
    print("Row for SALES_ORDER_ID 6383668:\n", filtered_df)

Row for SALES_ORDER_ID 6383668:
        PRODUCT_SALES_ORDER_ID  SALES_ORDER_ID  PRODUCT_ID    SALES
18757                67805080         6418568        9081  3521.25


In [52]:
# It turns out that the loss of ten million was a corrupted record and the revenue of that order was 3521.25
# so we'll replace this record with the correct value

# Step 1: Calculate the sum of SALES for the specific SALES_ORDER_ID
sales_sum = filtered_df['SALES'].sum()

# Step 2: Locate the row in sales_orders with the matching SALES_ORDER_ID
sales_orders.loc[sales_orders['ORDER_ID'] == order_id_to_find, 'SALES'] = sales_sum

# Display the updated sales_orders DataFrame to verify the change
print(sales_orders[sales_orders['ORDER_ID'] == order_id_to_find])

      ORDER_ID       DATE DAY_NAME Order_status    SALES
4713   6418568 2023-10-06      Fri    Delivered  3521.25


In [53]:
# double CHecking for outliers in SALES col of sales_orders
column_name = 'SALES'

min_val = sales_orders[column_name].min()
max_val = sales_orders[column_name].max()

print(f"Minimum value in {column_name}: {min_val}")
print(f"Maximum value in {column_name}: {max_val}")

Minimum value in SALES: 16.0
Maximum value in SALES: 9568.75


## 2. Cleaning Dataset 2 (Cost_of_Goods)

In [54]:
#Identify columns with missing values (NaN or null).
missing_values_columns = cost_of_goods.isna().any()
sum_missing_values_columns= cost_of_goods.isna().sum()
columns_with_missing_values = missing_values_columns[missing_values_columns].index.tolist()
print("Columns with missing values:", columns_with_missing_values, " \n ,Sum of missing values: \n",sum_missing_values_columns)

Columns with missing values: []  
 ,Sum of missing values: 
 product_id        0
purchase_price    0
selling_price     0
PRODUCT_ID        0
dtype: int64


In [55]:
#explore the cost of goods data
print(cost_of_goods.describe())


        product_id  purchase_price  selling_price   PRODUCT_ID
count    26.000000    2.600000e+01      26.000000    26.000000
mean   3277.230769    3.846339e+06     213.980769  3277.230769
std    3441.946209    1.961158e+07     142.814161  3441.946209
min      72.000000   -2.000000e+02      21.750000    72.000000
25%     458.500000    1.206600e+02     130.375000   458.500000
50%    1493.500000    1.823300e+02     189.750000  1493.500000
75%    6189.000000    2.699400e+02     281.000000  6189.000000
max    9468.000000    1.000000e+08     557.750000  9468.000000


In [56]:
# Identify rows with negative purchase_price
negative_values = cost_of_goods['purchase_price'] < 0

# Calculate the replacement value: selling_price - 11% * selling_price
replacement_value = cost_of_goods['selling_price'] - 0.11 * cost_of_goods['selling_price']

# Replace negative purchase_price values with the calculated replacement value
cost_of_goods.loc[negative_values, 'purchase_price'] = replacement_value[negative_values]
cost_of_goods

,product_id,purchase_price,selling_price,PRODUCT_ID
0,415,2.705600e+02,281.75,415
1,3495,2.001000e+01,21.75,3495
2,152,2.050800e+02,212.25,152
3,72,5.087900e+02,540.75,72
4,974,1.340100e+02,144.25,974
5,2438,4.089000e+01,43.00,2438
6,3900,5.019900e+02,557.75,3900
7,361,1.688775e+02,189.75,361
8,3575,2.680800e+02,282.25,3575
9,6952,1.448300e+02,155.50,6952


In [57]:
# detecting the other outliers in the purchasing price 

# Define the threshold as the highest selling_price
threshold = cost_of_goods['selling_price'].max()

# Identify rows with purchase_price greater than the threshold
large_value_rows = cost_of_goods['purchase_price'] > threshold

# Calculate the replacement value: selling_price - 11% * selling_price
replacement_value = cost_of_goods['selling_price'] - 0.11 * cost_of_goods['selling_price']

# Replace purchase_price values greater than the threshold with the calculated replacement value
cost_of_goods.loc[large_value_rows, 'purchase_price'] = replacement_value[large_value_rows]

cost_of_goods

,product_id,purchase_price,selling_price,PRODUCT_ID
0,415,270.5600,281.75,415
1,3495,20.0100,21.75,3495
2,152,205.0800,212.25,152
3,72,508.7900,540.75,72
4,974,134.0100,144.25,974
5,2438,40.8900,43.00,2438
6,3900,501.9900,557.75,3900
7,361,168.8775,189.75,361
8,3575,268.0800,282.25,3575
9,6952,144.8300,155.50,6952


## Q1 → calculate total sales (delivered orders only) per day.

- to answer this we need to filter out the cancelled orders 
- then calculate total sales per day

In [58]:
# Filter out canceled orders
delivered_orders = sales_orders[sales_orders['Order_status'] != 'Canceled']

# Group by both 'DATE' and 'DAY_NAME', then calculate total sales for delivered orders
total_sales_per_day = delivered_orders.groupby([delivered_orders['DATE'].dt.date, 'DAY_NAME'])['SALES'].sum().round(2)

total_sales_per_day

DATE        DAY_NAME
2023-10-01  Sun         371598.75
2023-10-02  Mon         320557.93
2023-10-03  Tue         449848.73
2023-10-04  Wed         437969.17
2023-10-05  Thu         225666.14
2023-10-06  Fri         231573.77
2023-10-07  Sat         342714.68
2023-10-08  Sun         393627.03
2023-10-09  Mon         313617.20
2023-10-10  Tue         275830.06
2023-10-11  Wed         324794.78
2023-10-12  Thu         234056.88
2023-10-13  Fri         173854.80
2023-10-14  Sat         277795.64
2023-10-15  Sun         264212.10
2023-10-16  Mon         290943.01
2023-10-17  Tue         232227.59
2023-10-18  Wed         298317.21
2023-10-19  Thu         254330.92
2023-10-20  Fri         203526.50
2023-10-21  Sat         281832.08
Name: SALES, dtype: float64

## Q2 → Given the 3 weeks of data from sales orders predict the 4th-week (from 22nd to 28th of October) total sales (delivered only)

- Feature Engineering: Extract relevant features like week numbers.
- Weekly Sales Calculation: Calculate total weekly sales for the given data.
- Prediction Model: Use these calculations to predict the 4th-week sales.



In [59]:
# Extract week number from 'DATE'
sales_orders['WEEK_NUMBER'] = sales_orders['DATE'].dt.isocalendar().week

# Calculate total sales for each week
weekly_sales = sales_orders.groupby('WEEK_NUMBER')['SALES'].sum().reset_index()

# Assuming the data contains only the first 3 weeks and you want to predict the 4th week
X = weekly_sales['WEEK_NUMBER'].values.reshape(-1, 1)  # Week numbers as features
y = weekly_sales['SALES'].values  # Sales as target

# Train a simple linear regression model
model = LinearRegression()
model.fit(X, y)

# # Predict the 4th week (assuming weeks are consecutive and the first week number is correct)
# predicted_sales_4th_week = model.predict([[X[-1] + 1]])
# Correcting the prediction line to ensure X is a 2D array
predicted_sales_4th_week = model.predict([[X[-1][0] + 1]])

print(f"Predicted total sales for the 4th week: {predicted_sales_4th_week[0]:.2f}")

Predicted total sales for the 4th week: 2539431.08


## Q3 → Based on your prediction in the previous question, what will be the contribution of Wednesday and Friday in the 4th week (from 22nd to 28th of October).

- To estimate the contribution of Wednesday and Friday in the 4th week based on the previous prediction, we need to follow these steps:

- Calculate the Proportion of Sales for Wednesday and Friday: Analyze the historical data to find the proportion of sales that occurred on Wednesday and Friday compared to the total weekly sales.
- Apply Proportions to the 4th Week's Prediction: Use the calculated proportions to estimate the sales for Wednesday and Friday in the 4th week.

In [60]:
# Placeholder for the predicted total sales for the 4th week
predicted_sales_4th_week = 100  # Example value

# Calculate the proportion of sales for Wednesday and Friday
# First, filter the data for only Wednesday and Friday
wed_fri_sales = sales_orders[sales_orders['DAY_NAME'].isin(['Wed', 'Fri'])]

# Calculate total sales for Wednesday and Friday
total_wed_fri_sales = wed_fri_sales['SALES'].sum()

# Calculate total sales for all days in the dataset
total_sales = sales_orders['SALES'].sum()

# Calculate the proportion of Wednesday and Friday sales to total sales
proportion_wed_fri = total_wed_fri_sales / total_sales

# Apply the proportion to the 4th week's prediction
predicted_wed_fri_sales_4th_week = predicted_sales_4th_week * proportion_wed_fri

# Assuming equal distribution between Wednesday and Friday
predicted_wed_sales = predicted_wed_fri_sales_4th_week / 2
predicted_fri_sales = predicted_wed_fri_sales_4th_week / 2

print(f"Predicted Wednesday sales for the 4th week: {predicted_wed_sales:.2f}")
print(f"Predicted Friday sales for the 4th week: {predicted_fri_sales:.2f}")

Predicted Wednesday sales for the 4th week: 13.53
Predicted Friday sales for the 4th week: 13.53


## Q4 → Based on the 4 weeks of data, Which week has the highest sales and which day is usually the highest per week in terms of sales (delivered only)

- To determine which week has the highest sales and which day is usually the highest per week in terms of sales (considering only delivered orders), follow these steps:

- Filter Delivered Orders: Since we're only interested in delivered orders, filter the dataset accordingly.
- Calculate Weekly Sales: Sum up sales for each week to find the week with the highest sales.
- Identify Highest Sales Day Per Week: For each week, find the day with the highest sales.

In [61]:
# Assuming 'sales_orders' DataFrame is already filtered for delivered orders

# Calculate total sales for each week
weekly_sales_totals = sales_orders.groupby('WEEK_NUMBER')['SALES'].sum()

# Find the week with the highest sales
highest_sales_week = weekly_sales_totals.idxmax()
highest_sales_amount = weekly_sales_totals.max()

print(f"Week with the highest sales: \n Week {highest_sales_week}, with Total Sales: {highest_sales_amount}")

# Calculate the day with the highest sales per week
highest_sales_day_per_week = sales_orders.groupby(['WEEK_NUMBER', 'DAY_NAME'])['SALES'].sum().reset_index()

# Sort to find the day with the highest sales per week
highest_sales_day_per_week = highest_sales_day_per_week.sort_values(by=['WEEK_NUMBER', 'SALES'], ascending=[True, False])

# Drop duplicates to keep only the top day per week
highest_sales_day_per_week = highest_sales_day_per_week.drop_duplicates(subset=['WEEK_NUMBER'], keep='first')

print("Highest sales day per week: \n")
print(highest_sales_day_per_week[['WEEK_NUMBER', 'DAY_NAME', 'SALES']])

Week with the highest sales: 
 Week 40, with Total Sales: 2626643.1246
Highest sales day per week: 

    WEEK_NUMBER DAY_NAME        SALES
0            39      Sun  404328.5020
7            40      Wed  494512.5500
9            41      Mon  357130.9463
20           42      Wed  337260.7745


## Given Dataset2 (Cost_of_Goods) and Dataset3 (Product_sales_order)
## Q5 → Calculate the margin percentage per product

In [62]:
#to calculate margin % Per product, we've to merge the two dfs into one combined df on the 
#on the common key which is (PRODUCT_ID).
# Join the dataframes based on PRODUCT_ID
cost_of_goods['PRODUCT_ID'] = cost_of_goods['product_id'].astype(int)

combined_df = pd.merge(cost_of_goods, product_sales_orders, on='PRODUCT_ID', how='inner')

# Calculate the margin percentage
combined_df['Margin Percentage'] = ((combined_df['selling_price'] - combined_df['purchase_price']) / combined_df['selling_price']) * 100

# Display the result
print(combined_df[['PRODUCT_ID', 'Margin Percentage']])


       PRODUCT_ID  Margin Percentage
0             415           3.971606
1             415           3.971606
2             415           3.971606
3             415           3.971606
4             415           3.971606
...           ...                ...
20273        8120           7.586481
20274        8120           7.586481
20275        8120           7.586481
20276        8120           7.586481
20277        8120           7.586481

[20278 rows x 2 columns]


## Q6 → Calculate the gross profit per product.
- basically, the Gross Profit = Selling Price - Purchase Price , so we'll use the combined df that we've allready created again and create a new col with this calculation 



In [63]:
# Calculate the gross profit
combined_df['Gross Profit'] = combined_df['selling_price'] - combined_df['purchase_price']

# Display the result
print(combined_df[['PRODUCT_ID', 'Gross Profit']])

       PRODUCT_ID  Gross Profit
0             415         11.19
1             415         11.19
2             415         11.19
3             415         11.19
4             415         11.19
...           ...           ...
20273        8120          9.54
20274        8120          9.54
20275        8120          9.54
20276        8120          9.54
20277        8120          9.54

[20278 rows x 2 columns]


## Q7 → What are the top 3 and bottom 3 products in terms of gross profit.

- to solve this we'll Sort and Identify Top/Bottom Products:
- We’ll sort the products based on their gross profit.
- The top 3 products will have the highest gross profit, and the bottom 3 products will have the lowest gross profit.

In [64]:
# Sort by gross profit (descending order)
sorted_df = combined_df.sort_values(by='Gross Profit', ascending=False)

# Get the top 3 and bottom 3 products
top_3_products = sorted_df.head(3)
bottom_3_products = sorted_df.tail(3)

# Display the results
print("Top 3 Products (Highest Gross Profit):")
print(top_3_products[['PRODUCT_ID', 'Gross Profit']])

print("\nBottom 3 Products (Lowest Gross Profit):")
print(bottom_3_products[['PRODUCT_ID', 'Gross Profit']])

Top 3 Products (Highest Gross Profit):
      PRODUCT_ID  Gross Profit
4518        3900         55.76
4421        3900         55.76
4423        3900         55.76

Bottom 3 Products (Lowest Gross Profit):
     PRODUCT_ID  Gross Profit
785        3495          1.74
784        3495          1.74
805        3495          1.74


## Q8 → State your recommendations for how we could further increase our gross profit.
- After we Identified which products contribute the most to gross profit. we should Focus on high-margin products (the top 3 highest products )
- also regarding the Pricing Strateg, whe should Evaluate our pricing strategy and consider adjusting prices based on market demand and competitor pricing.
- Negotiate better terms with suppliers to reduce purchase costs

## Q9 → If the 3 datasets (Sales_orders, Cost_of_Goods & Product_sales_order) were tables in database, write a simple SQL query to:
- create 1 table that has the total number of unique delivered products
- and the total number of unique canceled products per day for the first week 
- and create a flag that is equal to 1 if that day had more than than 5 unique products delivered and 0 else wise.
- Also make sure your answer doesn't contain fully duplicated rows

```sql

CREATE TABLE ProductSummary AS
SELECT
    DATE,
    COUNT(DISTINCT CASE WHEN Order_status = 'Delivered' THEN PRODUCT_ID END) AS DeliveredProducts,
    COUNT(DISTINCT CASE WHEN Order_status = 'Canceled' THEN PRODUCT_ID END) AS CanceledProducts,
    CASE WHEN COUNT(DISTINCT CASE WHEN Order_status = 'Delivered' THEN PRODUCT_ID END) > 5
    THEN 1 ELSE 0 END AS HighProductCountFlag
FROM
    Sales_orders
    JOIN Product_sales_orders ON Sales_orders.ORDER_ID = Product_sales_orders.SALES_ORDER_ID
WHERE
    DATE BETWEEN '2024-10-01' AND '2024-10-07'
GROUP BY
    DATE;

```